In [ ]:
import subprocess
import os
import time
PYTHON_BIN = "/home/huangweigang/miniconda3/envs/gh/bin/python"

print(f"使用 Python：{PYTHON_BIN}")

# ====== 只运行你指定的脚本 ======
scripts_to_run = [
    # "DM_pipeline_1114原版fix_hkf_lr5e5.py",
    "single_1209_v4_2.py",




    
]

for script in scripts_to_run:
    log_file = f"{script}.log"

    print(f"\n后台运行：{script}")
    print(f"日志文件：{log_file}")

    run_cmd = f"nohup {PYTHON_BIN} {script} > {log_file} 2>&1 &"

    subprocess.Popen(
        run_cmd,
        shell=True,
        executable="/bin/bash"
    )

    print("等待 20 秒后启动下一个任务...")
    time.sleep(20)

print("\n🎉 选定的训练脚本已在后台启动！")
print("查看运行状态：ps -ef | grep DM_pipeline")


In [ ]:
import subprocess

target = "/home/huangweigang/miniconda3/envs/gh/bin/python"

# 获取当前所有 GPU 的 UUID ↔ index 对照表
gpu_map = {}
gpu_query = subprocess.run(
    ["nvidia-smi", "--query-gpu=index,uuid", "--format=csv,noheader,nounits"],
    capture_output=True, text=True
)
for line in gpu_query.stdout.strip().split("\n"):
    if not line.strip():
        continue
    index, uuid = [x.strip() for x in line.split(",")]
    gpu_map[uuid] = index

# 查询当前所有计算进程
result = subprocess.run(
    ["nvidia-smi", "--query-compute-apps=pid,gpu_uuid,gpu_name,used_memory", "--format=csv,noheader,nounits"],
    capture_output=True, text=True
)

print(f"{'List':<5} {'GPU':<5} {'PID':<10} {'Mem(MB)':<10} COMMAND")
print("-" * 80)
count = 0
for list, line in enumerate(result.stdout.strip().split("\n"), 1):
    if not line.strip():
        continue
    pid, gpu_uuid, gpu_name, mem = [x.strip() for x in line.split(",")]
    gpu_index = gpu_map.get(gpu_uuid, "?")
    try:
        cmd = subprocess.check_output(["ps", "-p", pid, "-o", "cmd="]).decode().strip()
        if target in cmd:
            print(f"{count:<5} {gpu_index:<5} {pid:<10} {mem:<10} {cmd}")
            count += 1
    except subprocess.CalledProcessError:
        continue


In [ ]:
subprocess.run(
    ["kill", "1586450"]
)

In [ ]:
import re
import os
import time
from datetime import datetime
from pathlib import Path
from IPython.display import clear_output  # 用于清空输出

# 定义目标路径
log_directory = Path("/data4/huangweigang/gh/method")





def calculate_time_difference(start_time_obj, end_time_obj):
    """
    计算两个 datetime 对象之间的时间差，并返回格式化后的小时:分钟:秒（%H:%M:%S）字符串。
    该函数会考虑跨日期的情况。

    :param start_time_obj: 开始时间（datetime 对象）
    :param end_time_obj: 结束时间（datetime 对象）
    :return: 格式化后的时间差字符串（%H:%M:%S）
    """
    # 计算时间差
    time_diff = end_time_obj - start_time_obj

    # 获取天数和秒数
    days = time_diff.days  # 获取跨天的天数
    seconds = time_diff.seconds  # 获取秒数

    # 计算小时、分钟、秒
    hours, remainder = divmod(seconds, 3600)  # 计算小时和剩余秒数
    minutes, seconds = divmod(remainder, 60)  # 计算剩余的分钟和秒数

    # 将天数转化为小时并加到总小时数
    total_hours = hours + days * 24

    # 格式化为 "%H:%M:%S"
    time_str = f"{total_hours:02}:{minutes:02}:{seconds:02}"

    return time_str


def extract_progress(log_directory):
    # 清空终端输出
    # os.system('cls' if os.name == 'nt' else 'clear')  # 清空屏幕
    # 清空Jupyter输出
    clear_output(wait=True)  # 清除上次输出的内容

    # 检查目标路径是否存在
    if log_directory.exists():
        # 递归查找所有的 .log 文件
        log_files = list(log_directory.rglob("*.log"))
        
        if log_files:
            print("找到以下 .log 文件：")
            for log_file in log_files:
                print(f"\n{log_file}")
                new_progress = None
                start_time_obj = None
                end_time_obj = None
                ifCompleted = None

                # 检索最新进度
                with open(log_file, 'r', encoding='utf-8') as f:
                    lines = f.readlines()
                    
                    # 反向查找包含 "🧩" 的最后一行
                    for line in reversed(lines):
                        if "completed_completed_completed_completed" in line or "Training completed!" in line:
                            ifCompleted = True
                            end_time_obj = start_time_obj


                        if "🧩" in line:
                            
                            match = re.search(r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})', line)

                            if match:
                                combined_time = match.group()
                                end_time_obj = datetime.strptime(combined_time, "%Y-%m-%d %H:%M:%S")
                                # print(f"当前时间: {end_time_obj}")

                            new_progress = line.strip()
                            end_time_obj 
                            break  # 一旦找到进度就退出循环                

                    for line in lines:
                        if "✅ Config saved to:" in line:
                            # 使用正则表达式提取日期时间部分
                            match = re.search(r'(\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2})', line)

                            if match:
                                # 提取日期部分和时间部分
                                date_part = match.group()[0:10]  # 例如 2025-12-11
                                # print(f"提取的日期是: {date_part}")

                                # match = match.group().replace('-', ':')
                                time_part = match.group()[11:19] # 例如 12-16-23
                                time_part = time_part.replace('-', ':')
                                # print(f"提取的时间是: {time_part}")

                                
                                # 合并日期和时间，转换为 datetime 对象
                                combined_time = f"{date_part} {time_part}"  # 例如 2025-12-11 12:16:23
                                start_time_obj = datetime.strptime(combined_time, "%Y-%m-%d %H:%M:%S")
                                # print(f"开始时间: {start_time_obj}")




                time_str  = calculate_time_difference(start_time_obj, end_time_obj)
                output_info = new_progress + f" | running {time_str}"
                if ifCompleted:
                    new_progress = "✅ Training completed"
                    output_info = new_progress + f" | run done {time_str}"
                print(f"{output_info}")
                print(f"✅{start_time_obj}")


        else:
            print("在该路径下没有找到任何 .log 文件。")
    else:
        print(f"路径 {log_directory} 不存在。")

while True:
    extract_progress(log_directory)
    print("\n等待 5 秒后刷新...\n")
    time.sleep(5)


找到以下 .log 文件：

/data4/huangweigang/gh/method/single_1209_v4.py.log
✅ Training completed | run done 24:49:54
✅2026-01-25 13:26:05

/data4/huangweigang/gh/method/single_1209_v4_2.py.log
✅ Training completed | run done 24:03:42
✅2026-01-25 13:27:33

等待 5 秒后刷新...



KeyboardInterrupt: 